### Boosting
Osnovna ideja pojacavanja je da se ansambl gradi rekurzivno, dodajuci model po model, pri cemu se svaki od modela trenira tako da sto bolje nadomesti greske trenutnog modela,odnosno da ga pojaca. <br>
Boosting pomaze u smanjenju greske pristrasnosti koja nastaje kada modeli ne uspevaju da uhvate sve relevantne trendove u podacima. Na primer, jedan klasifikator mozda nece moci tacno da predvidi klasu objekta, ali kada grupisemo vise slabih klasifikatora od kojih svaki progresivno uci od pogresno klasifikovanih instanci drugih, mozemo izgraditi jedan jak model.

<img src='boost.png'>

[AdaBoost](https://blog.paperspace.com/adaboost-optimizer/#:~:text=AdaBoost%20is%20an%20ensemble%20learning,turn%20them%20into%20strong%20ones.) ((Adaptive Boosting) je prvi i najpoznatiji algoritam pojacavanja.

AdaBoost ima za cilj kombinovanje vise slabih baznih modela da bi se napravio jedan jak. Bazni model moze biti bilo koji od osnovnih klasifikatora ili regresionih modela.
<br>
Sta je "slab" bazni model? <br>
Slab bazni model je onaj koji radi bolje od slucajnog pogadjanja, ali i dalje ima lose mere kvaliteta.
### Kako radi?
AdaBoost koristi sumu panjeva (Decision Stumps), a ne stabla. Panjevi su stabla koja imaju samo jedan cvor i dva lista.
Panj moze koristiti samo jedan atribut da donese odluku.

<img src='adaboost.png' width=900\>

Slab bazni model (npr. panj odluke) se pravi nad trening skupom na osnovu ponderisanih uzoraka. Na pocetku, svim uzorcima dajemo jednaku tezinu. Na primer, ako u skupu za treniranje postoji M instanci mogu im se pridruziti tezine 1/M. <br>
Kreiramo panj odluke za svaku varijablu i analiziramo greske koje je bazni model napravio i proporcionalno njima dajemo nove tezine instancama. Na ovaj nacin se model usmerava da obrati vise paznje na instance sa vecim greskama ne bi li postao precizniji. <br>
Ceo postupak se ponavlja dok se ne dosegne zadati broj modela ansambla. 

### Implementacija

Implementiracemo AdaBoost boosting algoritam za predikciju cena nekretnina u Boston bazi podataka sto znaci da cemo koristiti regresioni bazni model kako imamo kontinualnu ciljnu promenljivu.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn import model_selection
from sklearn import datasets
from sklearn import preprocessing
from sklearn import metrics

In [ ]:
data = datasets.load_boston()

In [ ]:
print(data.DESCR)

In [ ]:
X = data.data
y = data.target

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn import tree
from sklearn import ensemble

Kao bazni model AdaBoost algoritma koristimo regresiono stablo sa maksimalnom dubinom 3. <br>
U ansamblu imamo 100 stabala. <br>

In [ ]:
model_adaboost= ensemble.AdaBoostRegressor(base_estimator=tree.DecisionTreeRegressor(max_depth=3), n_estimators=100, random_state=42)

In [ ]:
model_adaboost.fit(X_train, y_train)

In [ ]:
y_pred = model_adaboost.predict(X_test)

In [ ]:
metrics.mean_squared_error(y_test, y_pred) # uporediti sa linearnom regresijom

In [ ]:
metrics.r2_score(y_test, y_pred)

Grafik promene gresaka baznih modela:

In [ ]:
plt.plot(range(0, model_adaboost.n_estimators), model_adaboost.estimator_errors_)
plt.show()

Grafik promene tezina baznih modela (sto je bolji model on daje vece tezine pogresnim predikcijama tj. manje tacnim):

In [ ]:
plt.plot(range(0, model_adaboost.n_estimators), model_adaboost.estimator_weights_)
plt.show()

In [ ]:
scores= []
for estimator in model_adaboost.estimators_:
    score = estimator.score(X_train, y_train)
    scores.append(score)

In [ ]:
plt.title('Promena koeficijenta determinacije u toku treniranja')
plt.xlabel('Redni broj estimatora')
plt.ylabel('R2')
plt.plot(range(0, model_adaboost.n_estimators), scores)
plt.show()

Znacajnost atributa:

In [ ]:
plt.barh(data.feature_names, model_adaboost.feature_importances_, color='orange') # uporediti sa linearnom regresijom
plt.show()